In [3]:
import numpy as np 
from datasets import load_dataset, DatasetDict, Dataset
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.preprocessing import OneHotEncoder
from tokenwiser.pipeline import make_partial_union

In [4]:
ds = load_dataset('silicone', 'dyda_da')

Reusing dataset silicone (/home/vincent/.cache/huggingface/datasets/silicone/dyda_da/1.0.0/af617406c94e3f78da85f7ea74ebfbd3f297a9665cb54adbae305b03bc4442a5)


In [19]:
import pandas as pd 

pd.concat([
    ds['train'].to_pandas()[['Utterance', 'Label']].assign(split="train"),
    ds['validation'].to_pandas()[['Utterance', 'Label']].assign(split="valid")
]).rename(columns={'Utterance': 'text', 'Label': 'label'})

,text,label,split
0,"say , jim , how about going for a few beers af...",1,train
1,you know that is tempting but is really not go...,0,train
2,what do you mean ? it will help us to relax .,3,train
3,do you really think so ? i don't . it will jus...,3,train
4,i guess you are right.but what shall we do ? i...,3,train
...,...,...,...
8064,"oh , it must be very precious . is it breakable ?",3,valid
8065,"no , if you take some care when you use them .",2,valid
8066,how much is it ?,3,valid
8067,two thousand .,2,valid


In [ ]:
class ClassificationDataset:
    def __init__(self, path, text_col='text', label_col='label'):
        dataf = pd.read_csv(path)
        self.train = dataf.loc[lambda d: d['split'] == 'train'].reset_index()
        self.valid = dataf.loc[lambda d: d['split'] == 'valid'].reset_index()
        self.labels = list(dataf[label_col].unique())
    
    def batch(self, n):
        indices = np.random.randint(len(self), size=n)
        subset = self.train.iloc[indices]
        return subset[self.text_col], subset[self.label_col]
    
    def full(self, split="train"):
        subset = self.train if split == "train" else self.valid
        return subset[self.text_col], subset[self.label_col]
    

class Batcher:
    def __init__(self, dataset, tokeniser):
        self.dataset = dataset
        self.tokeniser = tokeniser
        label_arr = np.array(self.dataset.labels).reshape(-1, 1)
        self.label_enc = OneHotEncoder(sparse=False).fit(label_arr)
    
    def batch(self, n):
        text, labs = self.dataset.batch(n=n)
        return self.tokeniser.transform(text), self.label_enc.transform(labs)
    
    def full(self, split="train"):
        text, labs = self.dataset.full(split=split)
        return self.tokeniser.transform(text), self.label_enc.transform(labs)
    
    def transform(self, X):
        return self.tokeniser.transform(text)

In [24]:
class TextDataset(Dataset):
    def __init__(self, name='silicone', subset='dyda_da', split='train', n_feat=20_000):
        self.dataset = load_dataset(name, subset)
        if isinstance(self.dataset, DatasetDict):
            self.dataset = self.dataset[split]
        self.labels = list(set(i['Label'] for i in self.dataset))
        self.name = f"{name}-{subset}-{split}"
        self.tfm = make_partial_union(
            HashingVectorizer(n_features=n_feat), 
            HashingVectorizer(n_features=n_feat, ngram_range=(2, 2))
        )
        self.label_enc = OneHotEncoder(sparse=False).fit(np.array(self.labels).reshape(-1, 1))

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        return item['Utterance'], item['Label']

    def __repr__(self):
        return f"<TextDataset {self.name}>"
    
    def batch(self, n):
        """Samples a random batch of `n` datapoints."""
        indices = np.random.randint(len(self), size=n)
        texts, labels = zip(*[self[int(i)] for i in indices])
        X = self.tfm.transform(texts)
        y = self.label_enc.transform(np.array(labels).reshape(-1, 1))
        return X, y
    
    def full(self):
        """Returns the full set in matrix form."""
        texts, labels = zip(*[self[int(i)] for i in range(len(self))])
        X = self.tfm.transform(texts)
        y = self.label_enc.transform(np.array(labels).reshape(-1, 1))
        return X, y
    
    def transform(self, texts):
        return self.tfm.transform(texts)

In [14]:
my_datasets = {d.name: {'dataset': d} for d in [
    TextDataset('silicone', 'dyda_da'),
    TextDataset('silicone', 'dyda_e'),
    TextDataset('silicone', 'meld_e')]
}

Reusing dataset silicone (/home/vincent/.cache/huggingface/datasets/silicone/dyda_da/1.0.0/af617406c94e3f78da85f7ea74ebfbd3f297a9665cb54adbae305b03bc4442a5)
Reusing dataset silicone (/home/vincent/.cache/huggingface/datasets/silicone/dyda_e/1.0.0/af617406c94e3f78da85f7ea74ebfbd3f297a9665cb54adbae305b03bc4442a5)
Reusing dataset silicone (/home/vincent/.cache/huggingface/datasets/silicone/meld_e/1.0.0/af617406c94e3f78da85f7ea74ebfbd3f297a9665cb54adbae305b03bc4442a5)


```python
fuse = (
    FUSE(tokeniser, n_tok_feat)
      .add_task(name, subset)
      .add_task(name, subset)
      .add_task(name, subset)
)
```

In [15]:
my_datasets

{'silicone-dyda_da-train': {'dataset': <TextDataset silicone-dyda_da-train>},
 'silicone-dyda_e-train': {'dataset': <TextDataset silicone-dyda_e-train>},
 'silicone-meld_e-train': {'dataset': <TextDataset silicone-meld_e-train>}}

In [22]:
from keras.layers import Dense, Input
from keras.models import Model
import scipy
import numpy as np

X, y = my_datasets['silicone-dyda_da-train']['dataset'].batch(1)
inputs = Input(shape=(X.shape[1],), sparse=True)
emb1 = Dense(256, activation='relu')(inputs)
emb2 = Dense(256, activation='relu')(emb1)

for dataset in my_datasets.values():
    X, y = dataset['dataset'].batch(8)
    dataset['outputs'] = Dense(y.shape[1], activation='softmax')(emb2)
    dataset['model'] = Model(inputs=inputs, outputs=dataset['outputs'])
    dataset['model'].compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [23]:
for epoch in range(100):
    for dataset in my_datasets.keys():
        X, y = my_datasets[dataset]['dataset'].batch(2048)
        print(dataset)
        my_datasets[dataset]['model'].fit(X, y, batch_size=128)

silicone-dyda_da-train


/home/vincent/Development/fuse/venv/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model_3/dense_8/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model_3/dense_8/embedding_lookup_sparse/Reshape:0", shape=(None, 256), dtype=float32), dense_shape=Tensor("gradient_tape/model_3/dense_8/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)


16/16 [==============================] - 2s 87ms/step - loss: 1.2872 - accuracy: 0.4609
silicone-dyda_e-train


/home/vincent/Development/fuse/venv/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model_4/dense_8/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model_4/dense_8/embedding_lookup_sparse/Reshape:0", shape=(None, 256), dtype=float32), dense_shape=Tensor("gradient_tape/model_4/dense_8/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)


16/16 [==============================] - 2s 88ms/step - loss: 1.2130 - accuracy: 0.8213
silicone-meld_e-train


/home/vincent/Development/fuse/venv/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model_5/dense_8/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model_5/dense_8/embedding_lookup_sparse/Reshape:0", shape=(None, 256), dtype=float32), dense_shape=Tensor("gradient_tape/model_5/dense_8/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)


16/16 [==============================] - 2s 86ms/step - loss: 1.7023 - accuracy: 0.4023
silicone-dyda_da-train
16/16 [==============================] - 1s 86ms/step - loss: 1.3760 - accuracy: 0.5303
silicone-dyda_e-train
16/16 [==============================] - 1s 87ms/step - loss: 0.7371 - accuracy: 0.8149
silicone-meld_e-train
16/16 [==============================] - 1s 87ms/step - loss: 1.5137 - accuracy: 0.4771
silicone-dyda_da-train
16/16 [==============================] - 1s 88ms/step - loss: 1.0666 - accuracy: 0.6353
silicone-dyda_e-train
16/16 [==============================] - 1s 88ms/step - loss: 0.5917 - accuracy: 0.8394
silicone-meld_e-train
16/16 [==============================] - 1s 86ms/step - loss: 1.4685 - accuracy: 0.4624
silicone-dyda_da-train
16/16 [==============================] - 1s 88ms/step - loss: 0.8818 - accuracy: 0.6685
silicone-dyda_e-train
16/16 [==============================] - 1s 90ms/step - loss: 0.5650 - accuracy: 0.8311
silicone-meld_e-train
16/16 [

In [ ]:
emb_model = Model(inputs=inputs, outputs=emb2)
emb_model.predict(